In [236]:
import pandas as pd
import re

In [3]:
phone_base = pd.read_excel('phone_base.xlsm')

In [5]:
phone_base.head()

,РЕГИОН,КОД,НУМЕРАЦИЯ,Начало,Конец,Нач_1,Кон_1,РЕГИОН.1
0,Ростовская область,901,0000000 по 0009999,0,9999,9010000000,9010009999,Ростовская область
1,Краснодарский край,901,1000000 по 1009999,1000000,1009999,9011000000,9011009999,Краснодарский край
2,Краснодарский край,901,1010000 по 1019999,1010000,1019999,9011010000,9011019999,Краснодарский край
3,Кабардино-Балкарская Республика,901,1040000 по 1049999,1040000,1049999,9011040000,9011049999,Кабардино-Балкарская Республика
4,Карачаево-Черкесская Республика,901,1050000 по 1059999,1050000,1059999,9011050000,9011059999,Карачаево-Черкесская Республика


In [6]:
column_names = ['Region', 'Code', 'useless', 'start', 'end', 'fullstart', 'fullend', 'Region-2']

In [7]:
phone_base.columns = column_names

In [8]:
phone_base.head()

,Region,Code,useless,start,end,fullstart,fullend,Region-2
0,Ростовская область,901,0000000 по 0009999,0,9999,9010000000,9010009999,Ростовская область
1,Краснодарский край,901,1000000 по 1009999,1000000,1009999,9011000000,9011009999,Краснодарский край
2,Краснодарский край,901,1010000 по 1019999,1010000,1019999,9011010000,9011019999,Краснодарский край
3,Кабардино-Балкарская Республика,901,1040000 по 1049999,1040000,1049999,9011040000,9011049999,Кабардино-Балкарская Республика
4,Карачаево-Черкесская Республика,901,1050000 по 1059999,1050000,1059999,9011050000,9011059999,Карачаево-Черкесская Республика


In [51]:
phone_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Region     2558 non-null   object
 1   Code       2558 non-null   int64 
 2   useless    2558 non-null   object
 3   start      2558 non-null   int64 
 4   end        2558 non-null   int64 
 5   fullstart  2558 non-null   int64 
 6   fullend    2558 non-null   int64 
 7   Region-2   2558 non-null   object
dtypes: int64(5), object(3)
memory usage: 160.0+ KB


In [54]:
phone_base.loc[3]['Region']

'Кабардино-Балкарская Республика'

### 1-й вариант

In [230]:
def get_region_by_phone(phone, phone_base):
    if type(phone) != str:
        phone = str(phone)
    phone = phone.replace('-', '').replace('(', '').replace(')', '').replace(' ', '')
    if phone[0] == '+':
        phone = phone[1:]
    if phone[0] == '8' or phone[0] == '7':
        phone = phone[1:]
    if len(phone) != 10:
        return 'Incorrect phone'
    return phone_base.loc[(phone_base['Code'] == int(phone[0:3])) & 
                          (phone_base['start'] <= int(phone[3:])) &
                          (phone_base['end'] >= int(phone[3:]))
                         ]['Region'].to_string(index=False).strip()

### 2-й вариант (re.sub)

In [243]:
def get_region_by_phone(phone, phone_base):
    if type(phone) != str:
        phone = str(phone)
    phone = re.sub('\D', '', phone)    
    if phone[0] == '8' or phone[0] == '7':
        phone = phone[1:]
    if len(phone) != 10:
        return 'Incorrect phone'
    return phone_base.loc[(phone_base['Code'] == int(phone[0:3])) & 
                          (phone_base['start'] <= int(phone[3:])) &
                          (phone_base['end'] >= int(phone[3:]))
                         ]['Region'].to_string(index=False).strip()

### 3-й вариант (через генератор)

In [253]:
def get_region_by_phone(phone, phone_base):
    if type(phone) != str:
        phone = str(phone)
    phone = ''.join(i for i in phone if i.isdigit())    
    if phone[0] == '8' or phone[0] == '7':
        phone = phone[1:]
    if len(phone) != 10:
        return 'Incorrect phone'
    return phone_base.loc[(phone_base['Code'] == int(phone[0:3])) & 
                          (phone_base['start'] <= int(phone[3:])) &
                          (phone_base['end'] >= int(phone[3:]))
                         ]['Region'].to_string(index=False).strip()

In [254]:
a = 9011009999
b = '+79260001133'
c = '8 9011040000'
d = '7(928)997-00-00'

In [255]:
get_region_by_phone(a, phone_base)

'Краснодарский край'

In [256]:
get_region_by_phone(b, phone_base)

'Московская область'

In [257]:
get_region_by_phone(c, phone_base)

'Кабардино-Балкарская Республика'

In [258]:
get_region_by_phone(d, phone_base)

'Ставропольский край'